## Cross Country Transmission Capacity Validation

### Imports

In [ ]:
import pypsa
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

### Load the PyPSA-Eur network

In [ ]:

n = pypsa.Network( "C:\\Users\\user\\Documents\\oet-ember\\validation\\notebooks\\optimized_network.nc")

### Load Ember data

In [ ]:
csv_path = r"C:\Users\user\Downloads\europe_interconnection_data (1)\Interconnectors\REF_NTC.csv"
csv_df = pd.read_csv(csv_path)

### calculate, compare and plot cross country capacities 

In [ ]:
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

csv_df = pd.read_csv(csv_path)

if 'country' not in n.buses.columns:
    print("Inferring country from bus indices...")
    n.buses['country'] = n.buses.index.map(lambda x: x.split()[0] if isinstance(x, str) else None)
    n.buses['country'] = n.buses['country'].str.upper()
    n.buses = n.buses.dropna(subset=['country'])

# Calculate cross-country capacities for Lines (AC transmission)

lines = n.lines.copy()
lines['country0'] = lines['bus0'].map(n.buses['country'])
lines['country1'] = lines['bus1'].map(n.buses['country'])
cross_border_lines = lines[(lines['country0'] != lines['country1']) & lines['country0'].notna() & lines['country1'].notna() & (lines['country0'].str.match(r'^[A-Z]{2}$')) & (lines['country1'].str.match(r'^[A-Z]{2}$'))]

capacities = defaultdict(float)
for _, row in cross_border_lines.iterrows():
    pair = frozenset({row['country0'], row['country1']})
    capacities[pair] += row['s_nom']

# Calculate cross-country capacities for Links 
links = n.links.copy()
if not links.empty:
    links['country0'] = links['bus0'].map(n.buses['country'])
    links['country1'] = links['bus1'].map(n.buses['country'])
    cross_border_links = links[(links['country0'] != links['country1']) & links['country0'].notna() & links['country1'].notna() & (links['country0'].str.match(r'^[A-Z]{2}$')) & (links['country1'].str.match(r'^[A-Z]{2}$'))]

    for _, row in cross_border_links.iterrows():
        pair = frozenset({row['country0'], row['country1']})
        capacities[pair] += row['p_nom']

# Convert to DataFrame
capacities_df = pd.DataFrame({
    'Country Pair': ['-'.join(sorted(pair)) for pair in capacities.keys()],
    'Model Capacity (MW)': list(capacities.values())
}).sort_values('Country Pair').reset_index(drop=True)

# Filter to only valid two-country pairs
capacities_df = capacities_df[capacities_df['Country Pair'].str.match(r'^[A-Z]{2}-[A-Z]{2}$')]

# Adjust model capacity to average bidirectional
capacities_df['Model Capacity (MW)'] = capacities_df['Model Capacity (MW)']

# Filter for 2024 only
csv_df = csv_df[csv_df['Year'] == 2024]

# Create undirected 'Country Pair' and average NTC_F and NTC_B for comparison
csv_df['Country Pair'] = csv_df.apply(lambda row: '-'.join(sorted([row['From'], row['To']])), axis=1)
aggregated_csv = csv_df.groupby('Country Pair').agg({'NTC_F': 'mean', 'NTC_B': 'mean'}).reset_index()
aggregated_csv['CSV Capacity (MW)'] = (aggregated_csv['NTC_F'] + aggregated_csv['NTC_B']) / 2  # Average for undirected comparison

# Compare model and Ember data
comparison_df = pd.merge(capacities_df, aggregated_csv[['Country Pair', 'CSV Capacity (MW)']], on='Country Pair', how='outer').fillna(0)
comparison_df['Difference (MW)'] = comparison_df['Model Capacity (MW)'] - comparison_df['CSV Capacity (MW)']

# Filter for focus countries: DE, NL, IT, PL, CZ, GR
focus_comparison = comparison_df[comparison_df['Country Pair'].str.contains('DE|NL|IT|PL|CZ|GR', na=False)]

# Output comparison
print("Comparison of Model vs CSV Capacities for Focus Countries:")
print(focus_comparison)

# Export to CSV
focus_comparison.to_csv("focus_countries_comparison.csv", index=False)
print("\nComparison exported to 'focus_countries_comparison.csv'")

# Bar graph for focus countries
focus_comparison.plot(x='Country Pair', y=['Model Capacity (MW)', 'CSV Capacity (MW)'], kind='bar', figsize=(14, 8), width=0.35)
plt.title("Comparison of Model vs CSV Interconnection Capacities (2025) for Focus Countries: DE, NL, IT, PL, CZ, GR")
plt.ylabel("Capacity (MW)")
plt.xlabel("Country Pair")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Type")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Total cross-border capacity from model
total_capacity = capacities_df['Model Capacity (MW)'].sum()
print(f"\nTotal cross-border capacity across Europe from model: {total_capacity:.2f} MW")

## Calculate Multi-Sector CO₂ Emissions

### Emissions Calculation Function – Detailed Explanation

This function calculates **carbon emissions** from an energy system model. It supports two setups: one where a **CO₂ bus** is defined in the network (used to track carbon flows), and one where it isn’t (using emission factors instead). It computes **gross emissions**, **negative emissions**, and **net emissions** by sector (e.g., electricity, heating, transport).

---

#### Step 1: Check for a CO₂ Bus

- The function first checks if a **CO₂ bus** exists in the network.
- If it does, it uses **flows into the CO₂ bus** to calculate emissions.
- If not, it uses **emission factors** associated with generators and links.

---

#### Step 2: Flow-Based Emission Calculation (with CO₂ Bus)

##### a) Links

- Iterates over all `Link` components (e.g., converters, CCS, DAC).
- For each port (port0 to port4):
  - If the port is connected to the CO₂ bus:
    - Extracts the power flow time series.
    - Adds the **negative flows** (emissions into the CO₂ bus) to total emissions.
- Emissions are multiplied by snapshot weights to reflect actual contribution over time.

##### b) Generators

- Finds generators connected to the CO₂ bus.
- Extracts their power output time series.
- Sums the **positive flows** into the CO₂ bus as emissions.

##### c) Positive vs Negative Emissions

- **Positive emissions**: CO₂ released into the CO₂ bus (e.g., from coal/gas generators).
- **Negative emissions**: CO₂ captured or removed (e.g., from DAC or CCS).

##### d) Classify by Sector

Based on the `carrier` or `type`, emissions are grouped into sectors:

| Carrier Keywords        | Sector       |
|-------------------------|--------------|
| `coal`, `gas`, `lignite`| electricity  |
| `boiler`, `heat`, `CHP` | heating      |
| `vehicle`, `oil`        | transport    |
| `biomass`, `biogas`     | biomass      |
| `DAC`, `CCS`            | negative     |

##### e) Net Emissions

- **Net emissions** = Positive emissions – Negative emissions
- Values are summed and converted to **megatonnes of CO₂ equivalent (MtCO₂eq)**.

---

#### Step 3: Fallback Emission Calculation (without CO₂ Bus)

If no CO₂ bus is present:

##### a) Generators

- Uses a predefined **emission factor** (e.g., 0.34 t/MWh for natural gas).
- Multiplies each generator’s output by its emission factor.
- Adjusts for generator **efficiency** if available.
- Sums emissions over all snapshots.

##### b) Links

- Same approach as for generators.
- Uses the link’s `carrier` and input bus to look up the emission factor.
- Adjusts for efficiency where applicable.

##### c) Classify and Sum

- Emissions are classified by sector using the same method as above.
- Calculates **gross emissions**, **negative emissions**, and **net emissions**.

---

#### Step 4: CO₂ Store Override

If a **CO₂ Store** is modeled:

- Emissions stored via the store are extracted.
- This total **overrides** the calculated net emissions value.
- The result is used in place of previous net emissions if available.

---

#### Final Output

- Returns a **DataFrame** with sectoral emissions:
  - Rows: sectors (electricity, heating, etc.)
  - Columns: `gross_emissions`, `net_emissions`
- Also returns total emissions across all sectors in **MtCO₂eq**.




In [ ]:
def sector_emissions(n):
    # Snapshot weights for weighting sums
    weights = n.snapshot_weightings.objective

    # Define match_carrier helper
    def match_carrier(carrier, patterns):
        return pd.Series([carrier]).str.contains(patterns, case=False).item()

    # Check for CO2 bus
    co2_bus_candidates = ['CO2 atmosphere', 'co2 atmosphere', 'CO2']
    co2_bus = next((b for b in co2_bus_candidates if b in n.buses.index), None)

    if co2_bus:
        # CO2 flow-based calculation
        emissions_per_comp = {}

        # Links connected to CO2 bus
        for port in range(5):
            bus_col = f'bus{port}'
            if bus_col in n.links.columns:
                mask = n.links[bus_col] == co2_bus
                if mask.any():
                    links = n.links[mask]
                    p_col = f'p{port}'
                    if p_col in n.links_t:
                        p_port = n.links_t[p_col][links.index]
                        em_t = -p_port  # Emission = -flow at port (positive for emission)
                        em_weighted = em_t.mul(weights, axis=0).sum(axis=0)  # sum_t em_t * w_t per link
                        for link, em in em_weighted.items():
                            emissions_per_comp[link] = {
                                'em': em,
                                'type': 'Link',
                                'carrier': links.at[link, 'carrier']
                            }

        # Generators connected to CO2 bus (rare)
        mask_gen = n.generators.bus == co2_bus
        if mask_gen.any():
            gens = n.generators[mask_gen]
            p = n.generators_t.p[gens.index]
            em_t = p  # For generators, emission = flow to bus
            em_weighted = em_t.mul(weights, axis=0).sum(axis=0)
            for gen, em in em_weighted.items():
                emissions_per_comp[gen] = {
                    'em': em,
                    'type': 'Generator',
                    'carrier': gens.at[gen, 'carrier']
                }

        # Separate positive (gross) and negative (capture)
        pos_em = {k: v for k, v in emissions_per_comp.items() if v['em'] > 0}
        neg_em = {k: v for k, v in emissions_per_comp.items() if v['em'] < 0}

        # Define patterns
        power_patterns = 'coal|lignite|gas|OCGT|CCGT|oil|nuclear|CHP|power|gas boiler'
        transport_patterns = 'transport|vehicle|car|truck|bus|train|aviation|shipping|oil|diesel|petrol|kerosene|fossil'
        biomass_patterns = 'biomass|bio|waste|solid biomass|biogas'
        heating_patterns = 'heat|boiler|CHP'
        power_neg_patterns = 'power|elec|CHP|CCS'
        transport_neg_patterns = 'transport|aviation|shipping|CCS'
        biomass_neg_patterns = 'biomass|bio|waste|solid biomass|biogas|CCS'
        heating_neg_patterns = 'heat|boiler|CHP|CCS'

        # Assign positive emissions exclusively
        power_pos = 0
        transport_pos = 0
        biomass_pos = 0
        heating_pos = 0
        for v in pos_em.values():
            carrier = v['carrier']
            if match_carrier(carrier, transport_patterns):
                transport_pos += v['em']
            elif match_carrier(carrier, biomass_patterns):
                biomass_pos += v['em']
            elif match_carrier(carrier, heating_patterns):
                heating_pos += v['em']
            else:
                power_pos += v['em']

        # Assign negative emissions exclusively
        power_neg = 0
        transport_neg = 0
        biomass_neg = 0
        heating_neg = 0
        for v in neg_em.values():
            carrier = v['carrier']
            if match_carrier(carrier, transport_neg_patterns):
                transport_neg += v['em']
            elif match_carrier(carrier, biomass_neg_patterns):
                biomass_neg += v['em']
            elif match_carrier(carrier, heating_neg_patterns):
                heating_neg += v['em']
            else:
                power_neg += v['em']

        power_net = power_pos + power_neg
        transport_net = transport_pos + transport_neg
        biomass_net = biomass_pos + biomass_neg
        heating_net = heating_pos + heating_neg

        # Total net from calculations
        total_net_em = sum(v['em'] for v in emissions_per_comp.values()) / 1e6

    else:
        # Fallback to carrier-based if no CO2 bus
        # Generators
        gen_co2 = n.generators.carrier.map(n.carriers.co2_emissions)
        emitting_gens = n.generators[gen_co2 > 0]
        gen_p_weighted = n.generators_t.p.loc[:, emitting_gens.index].mul(weights, axis=0)
        gen_input = (gen_p_weighted / emitting_gens.efficiency).sum(axis=0)
        gen_emissions = gen_input * gen_co2[emitting_gens.index]

        # Links positive
        link_bus0_carrier = n.links.bus0.map(n.buses.carrier)
        link_co2 = link_bus0_carrier.map(n.carriers.co2_emissions)
        emitting_links = n.links[link_co2 > 0]
        link_p0_weighted = n.links_t.p0.loc[:, emitting_links.index].mul(weights, axis=0)
        link_input_pos = link_p0_weighted.sum(axis=0)
        link_emissions_pos = link_input_pos * link_co2[emitting_links.index]

        # Links negative
        negative_links = n.links[link_co2 < 0]
        link_p0_weighted_neg = n.links_t.p0.loc[:, negative_links.index].mul(weights, axis=0)
        link_input_neg = link_p0_weighted_neg.sum(axis=0)
        link_emissions_neg = link_input_neg * link_co2[negative_links.index]

        # Assign generator emissions exclusively
        power_pos = 0
        biomass_pos = 0
        for idx, em in gen_emissions.items():
            carrier = emitting_gens.at[idx, 'carrier']
            if match_carrier(carrier, 'biomass|bio|waste|solid biomass|biogas'):
                biomass_pos += em
            else:
                power_pos += em

        transport_pos = 0
        heating_pos = 0

        # Assign link positive emissions exclusively
        for idx, em in link_emissions_pos.items():
            carrier = emitting_links.at[idx, 'carrier']
            if match_carrier(carrier, 'transport|vehicle|car|truck|bus|train|aviation|shipping|oil|diesel|petrol|kerosene|fossil'):
                transport_pos += em
            elif match_carrier(carrier, 'biomass|bio|waste|solid biomass|biogas'):
                biomass_pos += em
            elif match_carrier(carrier, 'heat|boiler|CHP'):
                heating_pos += em
            else:
                power_pos += em

        # Assign link negative emissions exclusively
        power_neg = 0
        transport_neg = 0
        biomass_neg = 0
        heating_neg = 0
        for idx, em in link_emissions_neg.items():
            carrier = negative_links.at[idx, 'carrier']
            if match_carrier(carrier, 'transport|aviation|shipping|CCS'):
                transport_neg += em
            elif match_carrier(carrier, 'biomass|bio|waste|solid biomass|biogas|CCS'):
                biomass_neg += em
            elif match_carrier(carrier, 'heat|boiler|CHP|CCS'):
                heating_neg += em
            else:
                power_neg += em

        power_net = power_pos + power_neg
        transport_net = transport_pos + transport_neg
        biomass_net = biomass_pos + biomass_neg
        heating_net = heating_pos + heating_neg

        total_net_em = (gen_emissions.sum() + link_emissions_pos.sum() + link_emissions_neg.sum()) / 1e6

    # Override total with store if available
    if co2_bus:
        co2_stores = n.stores[n.stores.bus == co2_bus]
        if not co2_stores.empty:
            co2_store = co2_stores.index[0]
            e_initial = n.stores.at[co2_store, 'e_initial'] if 'e_initial' in n.stores.columns else 0
            total_net_em = (n.stores_t.e[co2_store].iloc[-1] - e_initial) / 1e6

    # DataFrame
    emissions_df = pd.DataFrame({
        'Sector': ['Power/Electricity', 'Transport', 'Biomass', 'Heating'],
        'Gross Emissions (MtCO2eq)': [power_pos / 1e6, transport_pos / 1e6, biomass_pos / 1e6, heating_pos / 1e6],
        'Net Emissions (MtCO2eq)': [power_net / 1e6, transport_net / 1e6, biomass_net / 1e6, heating_net / 1e6]
    })

    return emissions_df, total_net_em

In [ ]:
# Run the function
emissions_df, total_net_em = sector_emissions(n)

# Display results
display(emissions_df) 
print(f"Total Net Emissions: {total_net_em} MtCO2eq")